In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import pickle
from tqdm.notebook import tqdm

In [2]:
movies_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Movie_Recommendation_System/notebooks/preprocessing/preprocessed_dataset/preprocessed_dataset.csv')

In [3]:
movies_df.sample(5)

,id,title,overview
20914,233805,Bakhita,"born in a village in sudan, kidnapped by slave..."
15618,62981,Deadly Circuit,"beauvoir, lonely, aging private detective, is ..."
36577,271200,Heropanti,rising from present india in the conflicting e...
6032,41820,Till Human Voices Wake Us,"sam and silvy are best friends. one night, as ..."
14492,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."


In [4]:
movies_df.shape

(44475, 3)

In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44475 entries, 0 to 44474
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        44475 non-null  int64 
 1   title     44475 non-null  object
 2   overview  44475 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [6]:
movies_df.duplicated().sum()

0

In [7]:
movies_df.nunique()

id          44475
title       41370
overview    44473
dtype: int64

In [8]:
movies_df.isna().sum()

id          0
title       0
overview    0
dtype: int64

In [9]:
movies_df['title'] = movies_df['title'].apply(lambda x: x.lower())

In [ ]:
# !python -m spacy download en_core_web_lg

In [12]:
nlp = spacy.load('en_core_web_lg')

In [13]:
def get_lemma(overview):
  doc = nlp(overview)
  tokens = []
  for token in doc:
    if (token.is_alpha or token.like_num) and not token.is_stop:
      tokens.append(token.lemma_.lower())
  return ' '.join(tokens)

In [14]:
tqdm.pandas()

In [15]:
movies_df['overview'] = movies_df['overview'].progress_apply(get_lemma)

  0%|          | 0/44475 [00:00<?, ?it/s]

In [16]:
print(movies_df['id'].shape)
print(movies_df['overview'].shape)

(44475,)
(44475,)


In [17]:
id_lemma_list = list(zip(movies_df['id'],movies_df['overview']))
print('length: ',len(id_lemma_list))
for id,lemma in id_lemma_list[:5]:
  print(id,type(id),':',lemma)

length:  44475
862 <class 'int'> : lead woody andy toy live happily room andy birthday bring buzz lightyear scene afraid lose place andy heart woody plot buzz circumstance separate buzz woody owner duo eventually learn aside difference animation comedy family tomhank timallen donrickle jimvarney wallaceshawn johnlasseter jealousy toy boy friendship friend rivalry boy door new toy toy come life
8844 <class 'int'> : sibling judy peter discover enchanted board game open door magical world unwittingly invite alan adult trap inside game 26 year living room alan hope freedom finish game prove risky find run giant rhinoceros evil monkey terrifying creature adventure fantasy family robinwilliams jonathanhyde kirstendunst bradleypierce bonniehunt joejohnston board game disappearance base child book new home recluse giant insect
15602 <class 'int'> : family wedding reignite ancient feud door neighbor fishing buddy john max sultry italian divorcée open restaurant local bait shop alarm local worry

In [18]:
id_vector_map = {id:nlp(lemma).vector.reshape(1,-1) for id,lemma in tqdm(id_lemma_list)}

  0%|          | 0/44475 [00:00<?, ?it/s]

In [19]:
list(id_vector_map.items())[0][1].shape

(1, 300)

In [20]:
movies_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Movie_Recommendation_System/notebooks/preprocessing/preprocessed_dataset/preprocessed_text_dataset.csv',index=False)

In [21]:
with open('/content/drive/MyDrive/Colab Notebooks/Projects/Movie_Recommendation_System/config/id_vector_map.pkl','wb') as file:
  pickle.dump(id_vector_map,file)